In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import lightgbm as lgbm
# import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [2]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'

In [3]:
def read_feather(filename):
    df = pd.read_feather(os.path.join(INPUT_DIR, filename))
    print(filename, df.shape)
    return df

In [4]:
train_df = read_feather('train.f')
test_df = read_feather('test.f')

train.f (1997595, 35)
test.f (390095, 30)


In [5]:
whole_df = pd.concat([train_df, test_df], ignore_index=True)

In [6]:
campaign_df = read_feather('campaign.f')
map_gv_df = read_feather('map_game_feed_native_video_assets.f')
ad_video_df = read_feather('advertiser_video.f')
ad_cvideo_df = read_feather('advertiser_converted_video.f')

campaign.f (14627, 4)
map_game_feed_native_video_assets.f (2796, 3)
advertiser_video.f (11707, 6)
advertiser_converted_video.f (198622, 8)


In [7]:
ad_cvideo_df = ad_cvideo_df.drop_duplicates(
    subset=['mst_advertiser_video_id', 
                   'mst_game_feed_id', 
                    'mst_video_template_id'], keep='last')

le = preprocessing.LabelEncoder()
le.fit(['vertical', 'horizontal'])
ad_cvideo_df['rectangle_type_id'] = le.transform(ad_cvideo_df['rectangle_type'])
ad_cvideo_df.drop(columns=['rectangle_type'], inplace=True)
print('ad_cvideo_df', ad_cvideo_df.shape)

ad_cvideo_df (107493, 8)


### Label Encoding

In [8]:
label_encoding_col = ["adspot_id", "adspot_video_format_id", 
               "country_code", "game_feed_asset_type_id",
               "item_id", "os", "uid", "user_type_id", "video_template_id", "category_id", "game_feed_id", "game_template_id"]

# "advertiser_id", "app_id", "media_app_id", "campaign_id"

In [9]:
def get_non_overlapping(column: str):
    """train/testにしか出てこない値を調べる"""
    only_in_train = set(train_df[column].unique()) - set(test_df[column].unique())
    only_in_test = set(test_df[column].unique()) - set(train_df[column].unique())
    non_overlapping = only_in_train.union(only_in_test)
    return non_overlapping

def category2num(input_df, columns: list):
    input_ = input_df[columns].copy()
    for column in columns:
        non_overlapping = get_non_overlapping(column)
        if input_df[column].dtype == np.dtype("O"):
            # dtypeがobjectなら欠損は'missing' クラスにする
            input_[column] = input_df[column].fillna("missing")
            input_[column] = input_[column].map(lambda x: x if x not in non_overlapping else "other")
        else:
            # dtypeがint/floatなら欠損は'-1'とする
            input_[column] = input_df[column].fillna(-1)
            input_[column] = input_[column].map(lambda x: x if x not in non_overlapping else -2)

    return input_

In [10]:
train_LE = category2num(train_df, label_encoding_col)
print('train_LE', train_LE.shape)
test_LE = category2num(test_df, label_encoding_col)
print('test_LE', test_LE.shape)
concatenated = pd.concat([train_LE, test_LE], axis=0).reset_index(drop=True)

for column in label_encoding_col:
    le = preprocessing.LabelEncoder()
    le.fit(concatenated[column])
    train_LE[column] = le.transform(train_LE[column])
    test_LE[column] = le.transform(test_LE[column])

train_LE (1997595, 12)
test_LE (390095, 12)


In [11]:
def create_label_encoding_features(input_df, is_test=False):
    if is_test:
        return test_LE
    else:
        return train_LE

### target encoding

In [12]:
target_enc_col = ['advertiser_id', 'app_id', 'media_app_id', 'campaign_id']
# , 'category_id', 'game_feed_id', 'game_template_id', 'item_id', 'uid', 'user_type_id']

In [13]:
def target_encoding(train, test, enc_col):
    group_train = train.groupby([enc_col]).mean()[['target']].reset_index()
    test_copy = test[[enc_col]].copy()
    test_merge = pd.merge(test_copy, group_train, on=[enc_col], how='left')
    test_merge.set_index(test_copy.index, inplace=True)
    test_merge['target'].fillna(train['target'].mean(), inplace=True)
    enc_name = 'TGE_' + enc_col
    test_merge.rename(columns={'target': enc_name}, inplace=True)
    return test_merge.drop(columns=enc_col)

In [14]:
def create_targetencoding_features(input_df, is_test=False):
    
    if is_test:
        # test用 (全データ使用)
        print('for test')
        tgt_all = pd.DataFrame()
        for enc_col in target_enc_col:
            tmp = target_encoding(train_df, test_df, enc_col)
            tgt_all = pd.concat([tgt_all, tmp], axis=1)
        return tgt_all
    
    else:
        # train用 (oof)
        print('for train')
        kf = KFold(n_splits=5, shuffle=True, random_state=0)

        tgt_all = pd.DataFrame()
        for enc_col in target_enc_col:
            tgt_col = pd.DataFrame()

            for train_index, eval_index in kf.split(train_df):
                kf_train = train_df.iloc[train_index]
                kf_eval = train_df.iloc[eval_index]

                tmp = target_encoding(kf_train, kf_eval, enc_col)
                tgt_col = pd.concat([tgt_col, tmp])

            tgt_all = pd.concat([tgt_all, tgt_col], axis=1)
            print(enc_col)

        return tgt_all.sort_index()

### Count Encoding

In [15]:
def create_count_encoding_features(input_df, is_test=False):
    use_columns = [
        'category_id', 
        'adnw_id', 
        'adspot_id', 
        'app_id', 
        'advertiser_id',
        'uid', 
        'game_feed_id'
    ]

    out_df = pd.DataFrame()

    for c in use_columns:
        series = whole_df[c]
        vc = series.value_counts(dropna=False)

        _df = pd.DataFrame(input_df[c].map(vc))
        out_df = pd.concat([out_df, _df], axis=1)

    out_df = out_df.add_prefix('CE_')
    return out_df

### One-hot encoding

In [16]:
def create_onehot_encoding_features(input_df, is_test=False):
    use_columns = [
        # カーディナリティが比較的低めなものに絞っています
        'auction_type_id', 'header_bidding', 'is_interstitial', 'user_type_id' # and more
    ]

    out_df = pd.DataFrame()
    for c in use_columns:
        series = input_df[c]
        cat = pd.Categorical(series, categories=whole_df[c].dropna().unique())
        _df = pd.get_dummies(cat)
        _df.columns = _df.columns.tolist()
        _df = _df.add_prefix(c + '=')
        out_df = pd.concat([out_df, _df], axis=1)

    return out_df.add_prefix('OH_')

### 集約系

In [17]:
def create_aggregate_features(input_df, is_test=False):
    return _run_aggregation(input_df, 'uid')

In [18]:
def _run_aggregation(input_df, agg_column):
    _agg_df = pd.concat([
        whole_df.groupby(agg_column)['last_login_interval'].agg(['mean', 'min', 'max']).add_prefix('last_login_int_'),
        whole_df.groupby(agg_column)['first_login_interval'].agg(['mean', 'min', 'max']).add_prefix('first_login_int_'),
        whole_df.groupby(agg_column)['max_login_interval'].agg(['mean', 'min', 'max']).add_prefix('max_login_int_'),
        whole_df.groupby(agg_column)['frequency'].agg(['mean', 'min', 'max']).add_prefix('frequency_'),
        whole_df.groupby(agg_column)['login_frequency'].agg(['mean', 'min', 'max']).add_prefix('login_frequency_'),
        whole_df.groupby(agg_column)['advertiser_id'].nunique(),
        whole_df.groupby(agg_column)['app_id'].nunique(),
        whole_df.groupby(agg_column)['media_app_id'].nunique(),
        whole_df.groupby(agg_column)['campaign_id'].nunique()
    ], axis=1)

    out_df = pd.merge(input_df[agg_column], _agg_df, on=agg_column, how='left')
    out_df = out_df.drop(columns=agg_column).add_suffix('_by_{}'.format(agg_column))
    return out_df

### 四則演算

In [19]:
def create_operation_feature(input_df, is_test=False):
    out_df = pd.DataFrame(index=input_df.index)
    out_df['first_last_login_interval'] = input_df['first_login_interval'] - input_df['last_login_interval']
    out_df['max_last_login_interval'] = input_df['max_login_interval'] - input_df['last_login_interval']
    out_df['max_first_login_interval'] = input_df['max_login_interval'] - input_df['first_login_interval']
    out_df['login_freq_frequency'] = input_df['login_frequency'] - input_df['frequency']
    return out_df

### 連続変数

In [20]:
def create_continuous_features(input_df, is_test=False):
    use_columns = [
        # 連続変数
        'first_login_interval',
        'max_login_interval', 
        'frequency', 
        'login_frequency', 
        'last_login_interval',
        'from_click',
    ]
    return input_df[use_columns].copy()

### Category系

In [21]:
def create_category_features(input_df, is_test=False):
    use_columns = [
        'adnw_id',
        'header_bidding',
        'is_interstitial',
        'pos'
    ]
    return input_df[use_columns].copy()

### date系

In [22]:
def create_date_features(input_df, is_test=False):
    date_df = pd.DataFrame(pd.to_datetime(input_df['imp_at'], utc=True))
    date_df['imp_at'] = date_df['imp_at'].dt.tz_convert('Asia/Tokyo')
    date_df['day'] = date_df['imp_at'].dt.day
    date_df['hour'] = date_df['imp_at'].dt.hour
    date_df['total_minute'] = date_df['imp_at'].dt.hour*60+date_df['imp_at'].dt.minute
    date_df['dayofweek'] = date_df['imp_at'].dt.dayofweek
    date_df.drop(columns=['imp_at'], inplace=True)
    return date_df

#### campaign

In [23]:
def create_campaign_features(input_df, is_test=False):
    campaign = pd.merge(input_df[['campaign_id']], campaign_df, left_on='campaign_id', right_on='id', how='left')
    campaign.drop(columns=['campaign_id', 'id', 'mst_advertiser_id', 'mst_advertiser_order_id'], inplace=True)
    return campaign

#### map_game_feed_native_video_assets

In [24]:
def create_gamefeed_features(input_df, is_test=False):
    input_merge = pd.merge(input_df[['game_feed_id', 'advertiser_id', 'video_template_id']], map_gv_df, 
                           left_on='game_feed_id', right_on='mst_game_feed_id', how='left').drop(columns=['mst_game_feed_id'])
    
    horizontal = ad_video_df.copy()
    left_keys = ['horizontal_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    horizontal.columns = [f'horizontal_{c}' if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    vertical = ad_video_df.copy()
    left_keys = ['vertical_mst_advertiser_video_id', 'advertiser_id']
    right_keys = ['id', 'mst_advertiser_id']
    vertical.columns = [f'vertical_{c}' if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
    left_keys = [
        "horizontal_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    horizontal = ad_cvideo_df.copy()
    horizontal.columns = [f"horizontal_converted_{c}" if c not in right_keys else c for c in horizontal.columns]
    input_merge = pd.merge(input_merge, horizontal, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys) 
    
    left_keys = [
        "vertical_mst_advertiser_video_id",
        "game_feed_id",
        "video_template_id",
    ]
    right_keys = [
        "mst_advertiser_video_id",
        "mst_game_feed_id",
        "mst_video_template_id",
    ]
    vertical = ad_cvideo_df.copy()
    vertical.columns = [f"vertical_converted_{c}" if c not in right_keys else c for c in vertical.columns]
    input_merge = pd.merge(input_merge, vertical, left_on=left_keys, right_on=right_keys, how='left').drop(columns=right_keys)
    
    input_merge.drop(columns=['game_feed_id', 'advertiser_id', 'video_template_id', 
                              'horizontal_mst_advertiser_video_id', 'vertical_mst_advertiser_video_id'], inplace=True)
    
    # merge
    merge_col = ['duration', 'file_size', 'converted_file_size', 'converted_bitrate']
    vert_horz = ['vertical_', 'horizontal_']
    
    for m_col in merge_col:
        input_merge[m_col] = 0
        for vh in vert_horz:
            input_merge[m_col] = input_merge[m_col] + input_merge[vh+m_col].fillna(0)
            input_merge.drop(columns=[vh+m_col], inplace=True)
    
    return input_merge

In [25]:
processors = [
    create_continuous_features,
    create_category_features,
    create_label_encoding_features,
    create_date_features,
    create_campaign_features,
    create_gamefeed_features,
    create_targetencoding_features,
    create_count_encoding_features,
    create_onehot_encoding_features,
    create_aggregate_features,
    create_operation_feature
]

In [26]:
def to_feature(input_df, is_test=False):
    out_df = pd.DataFrame()
    for func in processors:
        _df = func(input_df, is_test)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return out_df

In [27]:
train_feat_df = to_feature(train_df)
test_feat_df = to_feature(test_df, True)
y = train_df['target']

for train
advertiser_id
app_id
media_app_id
campaign_id
for test


In [28]:
assert len(train_feat_df) == len(train_df)
assert len(test_feat_df) == len(test_df)

In [44]:
feature_count = len(train_feat_df.columns)
print(feature_count)

86


In [30]:
#del train_df, test_df

#### LightGBM による学習

In [31]:
from sklearn.metrics import average_precision_score

def pr_auc(y_pred, y_true):
    """lightGBM の round ごとに PR-AUC を計算する用"""
    score = average_precision_score(y_true.get_label(), y_pred)
    return "pr_auc", score, True

In [32]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'seed' : 0,
    'learning_rate':  0.1,
    'feature_pre_filter': False, 
    'lambda_l1': 6.419345380049121e-07, 
    'lambda_l2': 8.432801302426078, 
    'num_leaves': 212, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.9907178796872467, 
    'bagging_freq': 2, 
    'min_child_samples': 100
}

In [33]:
def train_lgbm(X, y, cv, params: dict, verbose=100):

    models = []
    # training data の target と同じだけのゼロ配列を用意
    # float にしないと悲しい事件が起こるのでそこだけ注意
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    num_boost_round=1000,
                                                    early_stopping_rounds=verbose,
                                                    feval=pr_auc,
                                                    verbose_eval=verbose)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

        print(f'Fold {i} PR-AUC: {average_precision_score(y_valid, y_pred):.4f}')

    score = average_precision_score(y, oof_pred)
    print('FINISHED \ whole score: {:.4f}'.format(score))
    return oof_pred, models, score

In [34]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cv = list(fold.split(train_feat_df, y)) # もともとが generator なため明示的に list に変換する

In [35]:
def tuning_lgbm(X, y, cv, params, verbose=100):
    idx_train, idx_valid = cv[0]
    x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    lgbm_train = lgbm.Dataset(x_train, y_train)
    lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
    
    best_params, tuning_history = dict(), list()
    best = lgbm.train(params,
                                  lgbm_train,
                                  valid_sets=lgbm_eval,
                                  num_boost_round=1000,
                                  early_stopping_rounds=verbose,
                                  feval=pr_auc,
                                  verbose_eval=0)
    print('Best Params:', best.params)
    print('Best Iteration:', best.best_iteration)
    print('Best Score:', best.best_score)

In [36]:
# %%time
# tuning_lgbm(train_feat_df, y, cv, params=lgbm_param)

In [37]:
%%time
oof, models, score = train_lgbm(train_feat_df, y, cv, params=lgbm_param)

[LightGBM] [Info] Number of positive: 52392, number of negative: 1545684
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.110880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8263
[LightGBM] [Info] Number of data points in the train set: 1598076, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.032784 -> initscore=-3.384468
[LightGBM] [Info] Start training from score -3.384468
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.0961149	valid_0's pr_auc: 0.321652
[200]	valid_0's binary_logloss: 0.0954468	valid_0's pr_auc: 0.327415
[300]	valid_0's binary_logloss: 0.0954021	valid_0's pr_auc: 0.327017
Early stopping, best iteration is:
[222]	valid_0's binary_logloss: 0.0953872	valid_0's pr_auc: 0.327866
Fold 0 PR-AUC: 0.3279
[LightGBM] [Info] Number of positive: 52

In [38]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [39]:
feature_importance(models)

,name,imp_0,imp_1,imp_2,imp_3,imp_4,sum
85,total_minute,1801,2035,2004,1618,2154,9612
83,TGE_campaign_id,1573,1967,1784,1341,1850,8515
84,CE_app_id,1591,1839,1748,1264,1785,8227
82,TGE_app_id,1517,1954,1662,1277,1800,8210
81,last_login_int_min_by_uid,1500,1916,1616,1230,1641,7903
...,...,...,...,...,...,...,...
4,OH_auction_type_id=4.0,2,3,3,1,3,12
3,OH_user_type_id=4,1,3,1,1,2,8
2,vertical_converted_width,0,0,0,0,0,0
1,vertical_converted_height,0,0,0,0,0,0


In [40]:
%%time
pred_list = []
for model in models:
    print('best_iteration', model.best_iteration)
    pred = model.predict(test_feat_df, num_iteration = model.best_iteration)
    pred_list.append(pred)
    
pred = np.mean(pred_list, axis=0)

best_iteration 222
best_iteration 280
best_iteration 249
best_iteration 189
best_iteration 261
CPU times: user 1min 55s, sys: 939 ms, total: 1min 56s
Wall time: 5.35 s


In [41]:
assert len(pred) == len(test_df)

In [42]:
sub_df = pd.DataFrame({ 'target': pred })
sub_df.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)

In [43]:
print('- feature={}'.format(feature_count))
print('- score={:.4f}'.format(score))

- feature=86
- score=0.3284


#### simple_9: 8-1, count, one-hot, aggregate, operation
- Wall time: 10min 48s
- feature=86
- score=0.3284
- publicLB= 0.2313 ★best★

#### simple_8-3: label->target : 'game_feed_id', 'game_template_id'
- feature=46
- score=0.2490
- publicLB= 0.2183

#### simple_8-2: label->target : category_id
- feature=46
- score=0.2492

#### simple_8-1: label->target : campaign_id
- feature=46
- score=0.2495

#### simple_7: 追加のtarget_encを抜いた
- Wall time: 7min 21s (vCPU x 24、メモリ 96 GB)
- feature=46
- score=0.2486
- publicLB= 0.2207

#### simple_6: label_enc + target_enc
- Wall time: 3min 56s (vCPU x 24、メモリ 96 GB)
- feature=55
- score=0.3255
- publicLB= 0.1798 (Overfit)

#### simple_5: target_enc (oof version)
- Wall time: 7min 20s
- feature=44
- score=0.2383
- publicLB= 0.2123

#### simple_4: target_enc
- Wall time: 8min
- feature=44
- score=0.2589
- publicLB= 0.1973 (leak)

#### simple_3: tuning
- Wall time: 7min 19s
- feature=41
- score=0.2229
- publicLB= 0.1970

#### simple_2
- Wall time: 47min 46s
- feature= 41
- score= 0.214588
- publicLB= 0.1907

#### tuning
```
Best Params: {
    'objective': 'binary', 
    'boosting_type': 'gbdt', 
    'seed': 0, 
    'learning_rate': 0.1, 
    'feature_pre_filter': False, 
    'lambda_l1': 6.419345380049121e-07, 
    'lambda_l2': 8.432801302426078, 
    'num_leaves': 212, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.9907178796872467, 
    'bagging_freq': 2, 
    'min_child_samples': 100, 
    'num_iterations': 1000, 
    'early_stopping_round': 100
}
Best Iteration: 245
Best Score: 'pr_auc', 0.22382995580267329
```